<a href="https://colab.research.google.com/github/Sharini-Jayabal/Transformer_TimeSeriesPrediction/blob/main/Transformer_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import math
import torch
from sklearn.preprocessing import MinMaxScaler


df = pd.read_csv('/content/energydata_complete.csv', index_col='date', parse_dates=['date'])
data = df['Appliances'].values.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(-1, 1))
data = scaler.fit_transform(data)

class TransformerModel(nn.Module):

    def __init__(self, input_size, hidden_size, num_classes, num_layers, num_heads, dropout):
        super(TransformerModel, self).__init__()

        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=input_size,
                nhead=num_heads,
                dropout=dropout
            ),
            num_layers=num_layers
        )
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        if len(x.shape) == 3:
            x = x.permute(1, 0, 2) 
        else:
            x = x.unsqueeze(1) 
            x = x.permute(1, 0, 2)
            x = self.transformer_encoder(x)
            x = x[-1, :, :] 
            x = self.fc(x)
        return x


train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]


output_size = 1
num_layers = 2
num_heads = 8  
hidden_size = 512
dropout = 0.1
lr = 0.001
num_epochs = 5
input_size = num_heads * 32
batch_size = 32



min_test_loss = float('inf')
best_trajectory = None


trajectories = ['Appliances', 'lights', 'T_out']
for traj in trajectories:

    traj_data = df[traj].values.reshape(-1, 1)
    traj_scaler = MinMaxScaler(feature_range=(-1, 1))
    traj_data = traj_scaler.fit_transform(traj_data)
    traj_train_data = traj_data[:train_size]
    traj_test_data = traj_data[train_size:]
    train_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(traj_train_data[:-1]), torch.FloatTensor(traj_train_data[1:]))
    test_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(traj_test_data[:-1]), torch.FloatTensor(traj_test_data[1:]))
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    train_loader.dataset.tensors = (
        train_loader.dataset.tensors[0].unsqueeze(0),
        train_loader.dataset.tensors[1].unsqueeze(0)
    )
    test_loader.dataset.tensors = (
        test_loader.dataset.tensors[0].unsqueeze(0),
        test_loader.dataset.tensors[1].unsqueeze(0)
    )

    model = TransformerModel(input_size, hidden_size, output_size, num_layers, num_heads, dropout)
    optimizer = optim.Adam(model.parameters(), lr=lr)

  
    for epoch in range(num_epochs):
        model.train()
        for i, (inputs, target) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            inputs.requires_grad_()
            targets.requires_grad_()
            loss = nn.MSELoss()(outputs, target)
            targets = targets.view(-1, 1, 1)
            #loss.backward()
            optimizer.step()

        with torch.no_grad():
            model.eval()
            test_loss = 0
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = nn.MSELoss()(outputs, targets)
                test_loss += loss.item()

            test_loss /= len(test_loader)
        print(f'Test loss for trajectory {traj}: {test_loss:.4f}')
        if test_loss < min_test_loss:
            min_test_loss = test_loss
            best_trajectory = traj
    print(f'Best trajectory so far: {best_trajectory}') 

       


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 15787, 1])) that is different to the input size (torch.Size([15787, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 3946, 1])) that is different to the input size (torch.Size([3946, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test loss for trajectory Appliances: 0.0578
Test loss for trajectory Appliances: 0.0578
Test loss for trajectory Appliances: 0.0578
Test loss for trajectory Appliances: 0.0578
Test loss for trajectory Appliances: 0.0578
Best trajectory so far: Appliances


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 15787, 1])) that is different to the input size (torch.Size([15787, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 3946, 1])) that is different to the input size (torch.Size([3946, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test loss for trajectory lights: 0.0488
Test loss for trajectory lights: 0.0488
Test loss for trajectory lights: 0.0488
Test loss for trajectory lights: 0.0488
Test loss for trajectory lights: 0.0488
Best trajectory so far: lights


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 15787, 1])) that is different to the input size (torch.Size([15787, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 3946, 1])) that is different to the input size (torch.Size([3946, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test loss for trajectory T_out: 0.2044
Test loss for trajectory T_out: 0.2044
Test loss for trajectory T_out: 0.2044
Test loss for trajectory T_out: 0.2044
Test loss for trajectory T_out: 0.2044
Best trajectory so far: lights
